In [136]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [137]:
NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'


METHOD = "STD-ICA"

In [138]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [139]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.004807,0.017778,-0.031672,0.010072,-0.034515,0.016089,0.012189,0.023488,-0.010707,...,0.023421,-0.020304,0.007167,0.023589,0.000907,0.002229,0.003083,0.006807,0.010904,-0.020461
1,CHUM-002,-0.006815,-0.017554,0.034618,-0.001564,0.015660,0.001713,0.012502,-0.007685,-0.008422,...,-0.021312,-0.016200,0.003548,0.002303,-0.019793,-0.021722,0.024258,-0.009981,-0.016233,0.038837
2,CHUM-006,0.006440,0.019804,-0.026084,0.012606,-0.018778,0.021355,-0.016724,0.010887,-0.018140,...,0.028220,-0.008982,0.016914,0.023810,0.003085,0.010984,-0.035439,0.019125,-0.015073,-0.067702
3,CHUM-007,0.000820,0.016962,-0.038353,0.012320,-0.004386,0.004073,-0.018448,0.020489,-0.022217,...,0.030868,0.007381,0.017927,0.016538,-0.011597,0.014322,-0.082704,0.032336,0.014617,-0.057481
4,CHUM-008,0.018374,0.023048,-0.036372,0.001714,-0.009185,0.006894,-0.016840,0.019799,0.003850,...,0.019988,-0.009583,-0.014454,0.026712,-0.009042,0.015153,-0.018142,0.022358,-0.012632,-0.015501


# Up to here

Read Outcome csv

In [140]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [141]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [142]:
OD.columns[0]

'Patient_ID'

In [143]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [144]:
OD.columns[0]


'Patient_ID'

In [145]:
PET_outcome_data.columns[0]

'Patient_ID'

In [146]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.004807,0.017778,-0.031672,0.010072,-0.034515,0.016089,0.012189,0.023488,-0.010707,...,0.023421,-0.020304,0.007167,0.023589,0.000907,0.002229,0.003083,0.006807,0.010904,-0.020461
1,CHUM-002,-0.006815,-0.017554,0.034618,-0.001564,0.015660,0.001713,0.012502,-0.007685,-0.008422,...,-0.021312,-0.016200,0.003548,0.002303,-0.019793,-0.021722,0.024258,-0.009981,-0.016233,0.038837
2,CHUM-006,0.006440,0.019804,-0.026084,0.012606,-0.018778,0.021355,-0.016724,0.010887,-0.018140,...,0.028220,-0.008982,0.016914,0.023810,0.003085,0.010984,-0.035439,0.019125,-0.015073,-0.067702
3,CHUM-007,0.000820,0.016962,-0.038353,0.012320,-0.004386,0.004073,-0.018448,0.020489,-0.022217,...,0.030868,0.007381,0.017927,0.016538,-0.011597,0.014322,-0.082704,0.032336,0.014617,-0.057481
4,CHUM-008,0.018374,0.023048,-0.036372,0.001714,-0.009185,0.006894,-0.016840,0.019799,0.003850,...,0.019988,-0.009583,-0.014454,0.026712,-0.009042,0.015153,-0.018142,0.022358,-0.012632,-0.015501


In [147]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [148]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [149]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,0.004807,0.017778,-0.031672,0.010072,-0.034515,0.016089,0.012189,0.023488,-0.010707,...,0.007167,0.023589,0.000907,0.002229,0.003083,0.006807,0.010904,-0.020461,0,1704
1,CHUM-002,-0.006815,-0.017554,0.034618,-0.001564,0.015660,0.001713,0.012502,-0.007685,-0.008422,...,0.003548,0.002303,-0.019793,-0.021722,0.024258,-0.009981,-0.016233,0.038837,1,439
2,CHUM-006,0.006440,0.019804,-0.026084,0.012606,-0.018778,0.021355,-0.016724,0.010887,-0.018140,...,0.016914,0.023810,0.003085,0.010984,-0.035439,0.019125,-0.015073,-0.067702,0,1186
3,CHUM-007,0.000820,0.016962,-0.038353,0.012320,-0.004386,0.004073,-0.018448,0.020489,-0.022217,...,0.017927,0.016538,-0.011597,0.014322,-0.082704,0.032336,0.014617,-0.057481,0,1702
4,CHUM-008,0.018374,0.023048,-0.036372,0.001714,-0.009185,0.006894,-0.016840,0.019799,0.003850,...,-0.014454,0.026712,-0.009042,0.015153,-0.018142,0.022358,-0.012632,-0.015501,0,1499


In [150]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)